In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime


# 날짜 데이터프레임 생성
data = {'Column1': [1, 2, 3], 'Column2': ['A', 'B', 'C']}
df = pd.DataFrame(data)

# 현재 날짜를 가져와 문자열로 변환
current_date = datetime.now().strftime("%Y%m%d")  # 'YYYYMMDD' 형식으로 날짜 생성

# 1. ChromeDriver 경로 설정
chrome_driver_path = 'C:\\P_Project\\chromedriver-win64\\chromedriver.exe'

# 2. WebDriver 설정 및 브라우저 열기
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# 3. 해당 URL 열기
url = 'https://www.kcar.com/bc/search'
driver.get(url)

# 4. 총 차량 개수 초기화
total_cars = 0

# 5. 차량 총 개수 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')
data = soup.select_one("div.subTitbox h2.subTitle span.textRed")

car_count = data.text.strip().replace(",", "")
car_count = int(car_count)
print(f"현재 페이지 차량 총 개수: {car_count}")

# 6. 36으로 나눈 몫 계산  27
quotient = car_count // 2700
print(f"27으로 나눈 몫: 총 {quotient}p")

# 차량 정보를 저장할 리스트
car_data = []

# 7. 페이지 루프
for page in range(quotient):  
    try:
        # 페이지 넘기기 전에 페이지 HTML을 가져옴
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.carListWrap"))
        )
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 페이지 넘기는 버튼을 기다렸다가 클릭
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, 'pageNext'))
        )
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(1)

        print(f"{page+1}p / {quotient}p")

        # 8. 각 차량 정보 추출
        for Count in range(27):
            # 차량 목록에서 div class="carListBox"를 찾고, adBannerBox 제외
            car_div = soup.select_one(f"div.carListWrap > div:nth-child({Count + 1})")

            # 페이지번호  ※다음 페이지로 이동할 수 없습니다
            pageNumbers = soup.select_one("li.pagingNum span")
            if car_div and "adBannerBox" not in car_div['class']:
                # 차량 명 
                car_name = car_div.select_one("div.detailInfo.srchTimedeal > div.carName > p")
                text_car_name = car_name.text.strip().replace(",", "") if car_name else ""
                # 가격
                price = car_div.select_one("div.detailInfo.srchTimedeal > div.carListFlex > div > p.carExp")
                only_price = 0
                if price:
                    price_text = price.text.strip()
                    match = re.search(r'(\d{1,3}(?:,\d{3})*)만원', price_text)  # 4,950 형식에 맞는 정규 표현식
                    if match:
                        only_price = int(match.group(1).replace(",", ""))

                # 자동차코드
                car_tag_code = car_div.select_one("p.timerInfoDtl.pointC")
                
                # 변수명 초기화
                code = None
                if car_tag_code and car_tag_code.has_attr('id'):
                    match = re.search(r'EC\d+', car_tag_code['id'])  # EC로 시작하는 숫자 추출
                    code = match.group() if match else None
                
                    car_url = (f'https://www.kcar.com/bc/detail/carInfoDtl?i_sCarCd={code}')
                
                # code가 None일 경우 img 태그에서 kcarM_ 코드 추출
                if code is None:
                    img_tag = car_div.select_one("a > img[alt='챠량이미지']")
                    if img_tag and img_tag.has_attr('src'):
                        src = img_tag['src']
                        
                        # src에서 이미지 URL만 추출
                        print("이미지 URL:", src)
                    else:
                        src = None

                                  
                # 년식
                year = car_div.select_one("div.detailInfo.srchTimedeal > div.carListFlex > p > span.block")
                text_year = int(re.search(r'\d+', year.text).group()) if year else ""

                # 주행 거리
                mileage = car_div.select_one("div.detailInfo.srchTimedeal > div.carListFlex > p > span:nth-child(2)")
                text_mileage = mileage.text.strip().replace(",", "") if mileage else ""

                # 연료
                fuel = car_div.select_one("div.detailInfo.srchTimedeal > div.carListFlex > p > span:nth-child(3)")
                text_fuel = fuel.text.strip() if fuel else ""

                # 위치
                location = car_div.select_one("div.detailInfo.srchTimedeal > div.carListFlex > p > span:nth-child(4)")
                text_location = location.text.strip() if location else ""

                # 차량 이미지 URL 추가
                car_image_url = None
                img_tag = car_div.select_one("a > img[alt='챠량이미지']")
                if img_tag and img_tag.has_attr('src'):
                    car_image_url = img_tag['src']  # 이미지 URL 그대로 가져오기

                # 차량 데이터를 딕셔너리로 저장
                car_info = {
                    "차량 이름": text_car_name,
                    "해당 URL" : car_url,
                    "가격": only_price,
                    "년식": text_year,
                    "주행 거리": text_mileage,
                    "연료": text_fuel,
                    "위치": text_location,
                    "차량 이미지 URL": car_image_url  # 차량 이미지 URL 추가
                }

                # 리스트에 추가
                car_data.append(car_info)

    except Exception as e:        
        print(text_car_name, car_url, only_price, text_year, text_mileage, text_fuel, text_location)
        print("다음 페이지로 이동할 수 없습니다:", str(e))
        break

# 9. pandas DataFrame으로 변환하여 엑셀 파일로 저장
df = pd.DataFrame(car_data)
df.to_excel(f"C:/P_Project/1.Project/5-Project_data/99.car_data_{current_date}.xlsx", index=False)

print("데이터가 엑셀 파일로 저장되었습니다.")

driver.quit()


현재 페이지 차량 총 개수: 9388
27으로 나눈 몫: 총 347p
1p / 347p
2p / 347p
3p / 347p
4p / 347p
5p / 347p
6p / 347p
7p / 347p
8p / 347p
9p / 347p
10p / 347p
11p / 347p
12p / 347p
13p / 347p
14p / 347p
15p / 347p
16p / 347p
17p / 347p
18p / 347p
19p / 347p
20p / 347p
21p / 347p
22p / 347p
23p / 347p
24p / 347p
25p / 347p
26p / 347p
27p / 347p
28p / 347p
29p / 347p
30p / 347p
31p / 347p
32p / 347p
33p / 347p
34p / 347p
35p / 347p
36p / 347p
37p / 347p
38p / 347p
39p / 347p
40p / 347p
41p / 347p
42p / 347p
43p / 347p
44p / 347p
45p / 347p
46p / 347p
47p / 347p
48p / 347p
49p / 347p
50p / 347p
51p / 347p
52p / 347p
53p / 347p
54p / 347p
55p / 347p
56p / 347p
57p / 347p
58p / 347p
59p / 347p
60p / 347p
61p / 347p
62p / 347p
63p / 347p
64p / 347p
65p / 347p
66p / 347p
67p / 347p
68p / 347p
69p / 347p
70p / 347p
71p / 347p
72p / 347p
73p / 347p
74p / 347p
75p / 347p
76p / 347p
77p / 347p
78p / 347p
79p / 347p
80p / 347p
81p / 347p
82p / 347p
83p / 347p
84p / 347p
85p / 347p
86p / 347p
87p / 347p
88p / 347p
89